### Spuštění PySpark

`export PYSPARK_PYTHON=python3`  
`pyspark --master yarn --num-executors 2 --executor-memory 4G --conf spark.ui.port=1<ddmm>`, kde `<ddmm>` je váš den a měsíc narození, např. `spark.ui.port=10811`

In [ ]:
# helpfull import
from pyspark.sql import functions as F

In [ ]:
#1.1
df = spark.read.csv('c:/code/fel/trips.txt', header=True)


In [ ]:
#1.2
df.cache()

In [ ]:
#1.3
df.show()

In [ ]:
#1.4
df.describe()

In [ ]:
#1.5
df.count()

In [ ]:
#1.6
df.select('route_id').distinct()
df.count()

In [ ]:
#1.7
df.select('route_id').groupBy().agg({'route_id':'max','route_id':'min'}).show()

In [ ]:
#1.8
df_l1=df.filter(df_routes['route_id'] == 'L1')
df_l1.groupBy('direction_id').count().show()

In [ ]:
#1.9
df_routes=spark.read.csv('c:/code/fel/routes.txt', header=True)
df_all =df.join(df_routes, df_routes['route_id']==df['route_id'])
df_all.filter(df_all['is_night'] == '1').count().show()

In [ ]:
# 2
songsDF = spark.read \
	.format("csv") \
	.option("header", "false") \
	.option("delimiter", ",") \
    .option("inferSchema", "true") \
	.load("/??/data/lyrics/lyrics.csv")



In [ ]:
#2.1

songsDF2 = songsDF.filter('rok>=1950 and rok<=2018')
songsDF2.cache() # uzitecne nakesovani, aby dalsi vypocty probihaly od tohoto bodu
songsDF2.count()


In [ ]:
#2.2

songsDF2 = songsDF2.fillna('', 'text')

songsDF2 = songsDF2.withColumn('text', F.lower(songsDF2['text']))

songsDF2 = songsDF2.withColumn('text', F.regexp_replace(songsDF2['text'], '[\W ]', ' '))

songsDF2 = songsDF2.withColumn('text', F.regexp_replace(songsDF2['text'], '[ ]+', ' '))

songsDF2 = songsDF2.withColumn('text', F.trim(songsDF2['text']))


In [ ]:
#2.4
songsDF2 = songsDF2.withColumn('slova_poc', F.size(F.split(songsDF2['text'], ' ')))
songsDF2.filter('text=""').show()
songsDF2 = songsDF2.withColumn('slova_poc', F.when(songsDF2['text']=='', 0).otherwise(songsDF2['slova_poc']))


In [ ]:
#2.4
songsDF2.cache()

In [ ]:
#3.1

interprets = songsDF2.groupBy('interpret').count() \
    .toDF('interpret', 'pocet').filter("pocet >= 500")

interprets.count()
interprets.show()


In [ ]:
#3.2 

songsDF2.filter('slova_poc > 0') \
    .groupBy('interpret').agg({'*':'count', 'slova_poc':'avg'}) \
    .toDF('interpret', 'prumer', 'pocet').filter('pocet>=100') \
    .orderBy('prumer', ascending=False) \
    .show()


In [ ]:
#4.1
words_top = songsDF2.rdd \
    .flatMap(lambda r: r[5].split(" ")) \
    .filter(lambda r: len(r)>1) \
    .map(lambda r: (r, 1)) \
    .reduceByKey(lambda a,b: a+b) \
    .sortBy(lambda r: r[1], False)
    
words_top.take(20)


stopw = sc.textFile("/??/data/stopwords.txt").collect()
stopw = set(stopw)

words_top2 = songsDF2.rdd \
    .flatMap(lambda r: r[5].split(" ")) \
    .filter(lambda r: len(r)>1) \
    .filter(lambda r: r not in stopw) \
    .map(lambda r: (r, 1)) \
    .reduceByKey(lambda a,b: a+b) \
    .sortBy(lambda r: r[1], False)
    
words_top2.take(20)


In [ ]:
#4.2

songsDF2 = songsDF2.withColumn('is_love', F.when(F.regexp_extract(songsDF2['text'], r'\b(love)\b', 1) == 'love', 1).otherwise(0))
songsDF2 = songsDF2.withColumn('is_like', F.when(F.regexp_extract(songsDF2['text'], r'\b(like)\b', 1) == 'like', 1).otherwise(0))
songsDF2 = songsDF2.withColumn('is_know', F.when(F.regexp_extract(songsDF2['text'], r'\b(know)\b', 1) == 'know', 1).otherwise(0))


In [ ]:
#4.3
interprets_words = interprets.join(songsDF2, 'interpret') \
    .select('interpret', 'is_love', 'is_like', 'is_know') \
    .groupBy('interpret') \
    .agg({'is_love':'avg', 'is_like':'avg', 'is_know':'avg'})

interprets_words.show()

